# LLaDA: 日本語プロンプト比較実験ノートブック

このノートブックでは、LLaDA（Large Language Diffusion with mAsking）を使って日本語プロンプトでの拡散言語モデルの動作を比較分析します。

## 📚 目次
1. [環境設定とモデル読み込み](#1-環境設定とモデル読み込み)
2. [日本語での基本的な生成実験](#2-日本語での基本的な生成実験)
3. [英語vs日本語の比較実験](#3-英語vs日本語の比較実験)
4. [日本語特有の現象の分析](#4-日本語特有の現象の分析)
5. [異なる日本語タスクでの性能比較](#5-異なる日本語タスクでの性能比較)
6. [まとめと考察](#6-まとめと考察)

## 1. 環境設定とモデル読み込み

In [ ]:
# Google Colabでの実行用
import sys

# Google Colabかどうかをチェック
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("Google Colab環境を検出しました")
    
    # 必要なライブラリをインストール
    !pip install transformers==4.49.0 accelerate==0.34.2 torch numpy matplotlib ipywidgets
    
    # GPUが利用可能かチェック
    import torch
    print(f"CUDA available: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"GPU: {torch.cuda.get_device_name(0)}")
        print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("ローカル環境で実行中です")

In [ ]:
# 必要なライブラリをインポート
import torch
import torch.nn.functional as F
import numpy as np
from transformers import AutoTokenizer, AutoModel
import matplotlib.pyplot as plt
import time
from typing import List, Tuple, Dict
import warnings
warnings.filterwarnings('ignore')

# 日本語フォント設定（Colab用）
if IN_COLAB:
    !apt-get -qq install fonts-noto-cjk
    import matplotlib.font_manager as fm
    plt.rcParams['font.family'] = ['DejaVu Sans', 'Noto Sans CJK JP']

print("ライブラリのインポートが完了しました")

In [ ]:
# LLaDAモデルとトークナイザーの読み込み
print("LLaDAモデルを読み込み中...（数分かかります）")

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"使用デバイス: {device}")

# モデルとトークナイザーの読み込み
model_name = 'GSAI-ML/LLaDA-8B-Instruct'
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModel.from_pretrained(
    model_name, 
    trust_remote_code=True, 
    torch_dtype=torch.bfloat16
).to(device).eval()

print("モデルの読み込みが完了しました！")
print(f"モデルサイズ: {sum(p.numel() for p in model.parameters()) / 1e9:.1f}B パラメータ")

In [ ]:
# 重要な定数とコア関数の定義
MASK_ID = 126336  # [MASK]トークンのID

def add_gumbel_noise(logits, temperature):
    """
    Gumbelノイズを追加してサンプリングを行う
    """
    if temperature == 0:
        return logits
    
    logits = logits.to(torch.float64)
    noise = torch.rand_like(logits, dtype=torch.float64)
    gumbel_noise = (-torch.log(noise)) ** temperature
    return logits.exp() / gumbel_noise

def get_num_transfer_tokens(mask_index, steps):
    """
    各ステップで何個のトークンをマスク解除するかを計算
    """
    mask_num = mask_index.sum(dim=1, keepdim=True)
    base = mask_num // steps
    remainder = mask_num % steps
    
    num_transfer_tokens = torch.zeros(
        mask_num.size(0), steps, 
        device=mask_index.device, 
        dtype=torch.int64
    ) + base
    
    for i in range(mask_num.size(0)):
        num_transfer_tokens[i, :remainder[i]] += 1
    
    return num_transfer_tokens

@torch.no_grad()
def generate_llada(model, prompt, steps=32, gen_length=64, block_length=32, 
                   temperature=0.0, cfg_scale=0.0, remasking='low_confidence'):
    """
    完全なLLaDA生成関数
    """
    # プロンプトをトークン化
    if isinstance(prompt, str):
        messages = [{"role": "user", "content": prompt}]
        formatted_prompt = tokenizer.apply_chat_template(
            messages, add_generation_prompt=True, tokenize=False
        )
        input_ids = tokenizer(formatted_prompt)['input_ids']
        input_ids = torch.tensor(input_ids).to(device).unsqueeze(0)
    else:
        input_ids = prompt
    
    prompt_length = input_ids.shape[1]
    
    # 応答部分を[MASK]で初期化
    x = torch.full(
        (1, prompt_length + gen_length), 
        MASK_ID, 
        dtype=torch.long
    ).to(device)
    x[:, :prompt_length] = input_ids.clone()
    
    # プロンプト部分のインデックス（CFG用）
    prompt_index = (x != MASK_ID)
    
    # ブロック処理の設定
    block_length = min(block_length, gen_length)
    num_blocks = (gen_length + block_length - 1) // block_length
    
    if steps % num_blocks != 0:
        adjusted_steps = ((steps + num_blocks - 1) // num_blocks) * num_blocks
        print(f"ステップ数を {steps} から {adjusted_steps} に調整しました")
        steps = adjusted_steps
    
    steps_per_block = steps // num_blocks
    generation_history = []
    
    print(f"生成開始: {num_blocks}ブロック x {steps_per_block}ステップ = 計{steps}ステップ")
    
    # 各ブロックを処理
    for block_idx in range(num_blocks):
        block_start = prompt_length + block_idx * block_length
        block_end = min(prompt_length + (block_idx + 1) * block_length, x.shape[1])
        
        print(f"\nブロック {block_idx + 1}/{num_blocks} (位置 {block_start}-{block_end-1}):")
        
        block_mask_index = (x[:, block_start:block_end] == MASK_ID)
        
        if not block_mask_index.any():
            print(f"  ブロック {block_idx + 1}: マスクなし、スキップ")
            continue
            
        num_transfer_tokens = get_num_transfer_tokens(block_mask_index, steps_per_block)
        
        for step in range(steps_per_block):
            mask_index = (x == MASK_ID)
            
            if not mask_index.any():
                print(f"  ステップ {step + 1}: 全トークン確定済み")
                break
            
            # 分類器フリーガイダンス (CFG)
            if cfg_scale > 0.0:
                un_x = x.clone()
                un_x[prompt_index] = MASK_ID
                x_ = torch.cat([x, un_x], dim=0)
                logits = model(x_).logits
                logits, un_logits = torch.chunk(logits, 2, dim=0)
                logits = un_logits + (cfg_scale + 1) * (logits - un_logits)
            else:
                logits = model(x).logits
            
            # Gumbelノイズでサンプリング
            logits_with_noise = add_gumbel_noise(logits, temperature)
            x0 = torch.argmax(logits_with_noise, dim=-1)
            
            # 再マスキング戦略に基づく信頼度計算
            if remasking == 'low_confidence':
                p = F.softmax(logits, dim=-1)
                x0_p = torch.squeeze(
                    torch.gather(p, dim=-1, index=torch.unsqueeze(x0, -1)), -1
                )
            elif remasking == 'random':
                x0_p = torch.rand((x0.shape[0], x0.shape[1]), device=x0.device)
            else:
                raise NotImplementedError(f"Remasking strategy '{remasking}' not implemented")
            
            x0_p[:, block_end:] = -float('inf')
            
            x0 = torch.where(mask_index, x0, x)
            confidence = torch.where(mask_index, x0_p, -float('inf'))
            
            # 高信頼度のトークンを選択してマスク解除
            transfer_index = torch.zeros_like(x0, dtype=torch.bool, device=device)
            for j in range(confidence.shape[0]):
                num_tokens_to_transfer = min(
                    num_transfer_tokens[j, step].item(),
                    mask_index[j].sum().item()
                )
                if num_tokens_to_transfer > 0:
                    _, select_index = torch.topk(confidence[j], k=num_tokens_to_transfer)
                    transfer_index[j, select_index] = True
            
            x[transfer_index] = x0[transfer_index]
            
            # 進捗を記録
            current_text = tokenizer.decode(
                x[0, prompt_length:], skip_special_tokens=False
            )
            avg_conf = confidence[0, transfer_index[0]].mean().item() if transfer_index[0].any() else 0
            
            generation_history.append({
                'block': block_idx,
                'step': step,
                'global_step': block_idx * steps_per_block + step,
                'text': current_text,
                'num_revealed': transfer_index[0].sum().item(),
                'avg_confidence': avg_conf
            })
            
            print(f"  ステップ {step+1:2d}: {transfer_index[0].sum().item():2d}個確定 "
                  f"(信頼度: {avg_conf:.3f})")
    
    # 最終結果
    final_text = tokenizer.decode(
        x[0, prompt_length:], skip_special_tokens=True
    )
    
    return final_text, generation_history

print("コア関数の定義が完了しました")

In [ ]:
# 拡張可視化関数の定義
def detect_token_changes(prev_tokens, curr_tokens):
    """
    前のステップと現在のステップでのトークン変化を検出
    """
    changes = []
    
    if prev_tokens is None:
        for i, token in enumerate(curr_tokens):
            if token != '[MASK]':
                changes.append((i, 'new', token))
    else:
        for i, (prev_token, curr_token) in enumerate(zip(prev_tokens, curr_tokens)):
            if prev_token != curr_token:
                if prev_token == '[MASK]' and curr_token != '[MASK]':
                    changes.append((i, 'revealed', curr_token))
                elif prev_token != '[MASK]' and curr_token == '[MASK]':
                    changes.append((i, 'masked', prev_token))
                elif prev_token != '[MASK]' and curr_token != '[MASK]':
                    changes.append((i, 'changed', curr_token))
    
    return changes

def get_confidence_color(confidence):
    """
    信頼度に基づいて落ち着いた色を決定
    """
    if confidence >= 0.8:
        return '\033[38;5;28m'  # 深い緑
    elif confidence >= 0.6:
        return '\033[38;5;94m'  # オリーブ
    elif confidence >= 0.4:
        return '\033[38;5;130m'  # ブラウン
    else:
        return '\033[38;5;240m'  # グレー

def visualize_generation_with_highlights(history, title="LLaDA Generation Process", show_details=True):
    """
    新しく生成された単語をハイライト表示する拡張可視化関数
    """
    print(f"\n{title}")
    print("=" * 70)
    
    if show_details:
        print("信頼度色分け:")
        print("  \033[38;5;28m■\033[0m 高信頼度 (≥0.8)  \033[38;5;94m■\033[0m 中信頼度 (≥0.6)  \033[38;5;130m■\033[0m 低信頼度 (≥0.4)  \033[38;5;240m■\033[0m 極低 (<0.4)")
        print("  強調: \033[1m太字\033[0m = 新規生成  \033[2m薄字\033[0m = 再マスク  \033[4m下線\033[0m = 変更")
        print()
    
    prev_tokens = None
    
    for i, entry in enumerate(history):
        global_step = entry.get('global_step', entry['step'])
        block = entry.get('block', 0)
        step = entry['step']
        text = entry['text']
        num_revealed = entry['num_revealed']
        avg_conf = entry['avg_confidence']
        
        clean_text = text.replace('<|reserved_special_token_250|>', '[MASK]')
        curr_tokens = clean_text.split()
        
        changes = detect_token_changes(prev_tokens, curr_tokens)
        
        block_info = f"[Block {block+1}]" if 'block' in entry else ""
        print(f"Step {global_step+1:2d} {block_info} (+{num_revealed:2d} tokens, conf={avg_conf:.3f}):")
        
        highlighted_text = []
        change_map = {pos: (change_type, token) for pos, change_type, token in changes}
        
        for pos, token in enumerate(curr_tokens):
            if pos in change_map:
                change_type, _ = change_map[pos]
                color = get_confidence_color(avg_conf)
                
                if change_type == 'revealed':
                    highlighted_text.append(f"{color}\033[1m{token}\033[0m")
                elif change_type == 'masked':
                    highlighted_text.append(f"\033[2m[MASK]\033[0m")
                elif change_type == 'changed':
                    highlighted_text.append(f"{color}\033[4m{token}\033[0m")
                else:
                    highlighted_text.append(f"{color}{token}\033[0m")
            else:
                if token == '[MASK]':
                    highlighted_text.append(f"\033[38;5;245m{token}\033[0m")
                else:
                    highlighted_text.append(token)
        
        print(f"  {' '.join(highlighted_text)}")
        
        if show_details and changes:
            change_summary = []
            for pos, change_type, token in changes:
                if change_type == 'revealed':
                    change_summary.append(f"pos{pos}:新規({token})")
                elif change_type == 'masked':
                    change_summary.append(f"pos{pos}:マスク")
                elif change_type == 'changed':
                    change_summary.append(f"pos{pos}:変更({token})")
            
            if change_summary:
                print(f"    変化: {', '.join(change_summary)}")
        
        print()
        prev_tokens = curr_tokens

def compare_generation_results(results, titles, detailed_analysis=True):
    """
    複数の生成結果を詳細に比較
    """
    print("生成結果比較分析")
    print("=" * 80)
    
    # 基本統計の比較
    print("\n基本統計:")
    print("-" * 40)
    for i, (result, title) in enumerate(zip(results, titles)):
        text, history = result
        avg_conf = np.mean([h['avg_confidence'] for h in history if h['avg_confidence'] > 0])
        total_steps = len(history)
        final_length = len(text.split())
        
        print(f"{title}:")
        print(f"  最終テキスト長: {final_length}語")
        print(f"  総ステップ数: {total_steps}")
        print(f"  平均信頼度: {avg_conf:.3f}")
        print(f"  最終結果: {text}")
        print()
    
    if detailed_analysis:
        print("\n詳細分析:")
        print("-" * 40)
        for i, (result, title) in enumerate(zip(results, titles)):
            text, history = result
            print(f"\n{title}の生成プロセス（最後の5ステップ）:")
            visualize_generation_with_highlights(history[-5:], f"{title}の詳細", show_details=False)

print("可視化関数の定義が完了しました")

## 2. 日本語での基本的な生成実験

In [ ]:
# 日本語での基本的な生成実験
japanese_prompts = [
    "日本の首都はどこですか？",
    "桜の季節について教えてください。",
    "おいしいラーメンの作り方を説明してください。"
]

print("=== 日本語での基本的な生成実験 ===")
japanese_results = []

for i, prompt in enumerate(japanese_prompts):
    print(f"\n{i+1}. プロンプト: {prompt}")
    print("-" * 50)
    
    result, history = generate_llada(
        model, prompt,
        steps=16, gen_length=64, block_length=32,
        temperature=0.0, cfg_scale=0.0, remasking='low_confidence'
    )
    
    japanese_results.append((result, history))
    print(f"\n生成結果: {result}")
    print("=" * 70)

In [ ]:
# 日本語生成プロセスの詳細可視化
print("=== 日本語生成プロセスの詳細分析 ===")

for i, (prompt, (result, history)) in enumerate(zip(japanese_prompts, japanese_results)):
    print(f"\n{i+1}. {prompt}")
    print("生成プロセス（最後の6ステップ）:")
    visualize_generation_with_highlights(
        history[-6:], 
        f"日本語生成プロセス {i+1}",
        show_details=True
    )

## 3. 英語vs日本語の比較実験

In [ ]:
# 対応する英語プロンプトで同じ実験
english_prompts = [
    "What is the capital of Japan?",
    "Please tell me about cherry blossom season.",
    "Please explain how to make delicious ramen."
]

print("=== 英語での対応実験 ===")
english_results = []

for i, prompt in enumerate(english_prompts):
    print(f"\n{i+1}. プロンプト: {prompt}")
    print("-" * 50)
    
    result, history = generate_llada(
        model, prompt,
        steps=16, gen_length=64, block_length=32,
        temperature=0.0, cfg_scale=0.0, remasking='low_confidence'
    )
    
    english_results.append((result, history))
    print(f"\n生成結果: {result}")
    print("=" * 70)

In [ ]:
# 日本語vs英語の詳細比較
print("=== 日本語 vs 英語 詳細比較 ===")

comparison_topics = [
    "日本の首都について",
    "桜の季節について", 
    "ラーメンの作り方について"
]

for i in range(len(japanese_prompts)):
    print(f"\n=== {comparison_topics[i]} ===")
    
    # 基本統計比較
    jp_text, jp_history = japanese_results[i]
    en_text, en_history = english_results[i]
    
    jp_avg_conf = np.mean([h['avg_confidence'] for h in jp_history if h['avg_confidence'] > 0])
    en_avg_conf = np.mean([h['avg_confidence'] for h in en_history if h['avg_confidence'] > 0])
    
    jp_tokens = len(jp_text.split())
    en_tokens = len(en_text.split())
    
    print(f"\n統計比較:")
    print(f"  日本語: {jp_tokens}語, 平均信頼度 {jp_avg_conf:.3f}")
    print(f"  英語:   {en_tokens}語, 平均信頼度 {en_avg_conf:.3f}")
    
    print(f"\n最終結果比較:")
    print(f"  日本語: {jp_text}")
    print(f"  英語:   {en_text}")
    
    # 生成プロセス比較（最後の4ステップ）
    print(f"\n生成プロセス比較（最後の4ステップ）:")
    print("\n日本語:")
    visualize_generation_with_highlights(
        jp_history[-4:], "日本語生成プロセス", show_details=False
    )
    
    print("\n英語:")
    visualize_generation_with_highlights(
        en_history[-4:], "英語生成プロセス", show_details=False
    )
    
    print("\n" + "=" * 80)

## 4. 日本語特有の現象の分析

In [ ]:
# 日本語特有の複雑な表現での実験
complex_japanese_prompts = [
    "『吾輩は猫である』の魅力について論じてください。",
    "日本の四季の美しさを俳句で表現してください。", 
    "関西弁で大阪の魅力を説明してください。",
    "敬語を使って会社での挨拶の仕方を教えてください。"
]

print("=== 日本語特有表現での生成実験 ===")
complex_results = []

for i, prompt in enumerate(complex_japanese_prompts):
    print(f"\n{i+1}. プロンプト: {prompt}")
    print("-" * 50)
    
    result, history = generate_llada(
        model, prompt,
        steps=20, gen_length=80, block_length=40,
        temperature=0.2, cfg_scale=0.0, remasking='low_confidence'
    )
    
    complex_results.append((result, history))
    print(f"\n生成結果: {result}")
    
    # 信頼度分析
    avg_conf = np.mean([h['avg_confidence'] for h in history if h['avg_confidence'] > 0])
    print(f"平均信頼度: {avg_conf:.3f}")
    print("=" * 70)

In [ ]:
# 日本語表現の複雑さと生成品質の関係分析
print("=== 日本語表現の複雑さ分析 ===")

# 各結果の詳細分析
analysis_labels = [
    "文学的表現",
    "韻律的表現（俳句）",
    "方言表現（関西弁）", 
    "敬語表現"
]

for i, ((result, history), label) in enumerate(zip(complex_results, analysis_labels)):
    print(f"\n=== {label} ===")
    
    # 統計情報
    avg_conf = np.mean([h['avg_confidence'] for h in history if h['avg_confidence'] > 0])
    conf_std = np.std([h['avg_confidence'] for h in history if h['avg_confidence'] > 0])
    total_steps = len(history)
    final_length = len(result.split())
    
    print(f"統計:")
    print(f"  最終長: {final_length}語")
    print(f"  ステップ数: {total_steps}")
    print(f"  平均信頼度: {avg_conf:.3f} (±{conf_std:.3f})")
    print(f"  結果: {result}")
    
    print(f"\n生成プロセス（最後の5ステップ）:")
    visualize_generation_with_highlights(
        history[-5:], f"{label}の生成プロセス", show_details=True
    )

## 5. 異なる日本語タスクでの性能比較

In [ ]:
# 異なるタイプの日本語タスクでの比較実験
task_experiments = {
    "数学問題": "12 + 15 = ? この計算を段階的に解いてください。",
    "料理レシピ": "卵焼きの作り方を簡潔に教えてください。",
    "科学説明": "光合成のしくみを分かりやすく説明してください。",
    "文化紹介": "日本の茶道の精神について説明してください。",
    "実用情報": "東京駅から羽田空港への行き方を教えてください。"
}

print("=== 異なる日本語タスクでの性能比較 ===")
task_results = {}

for task_name, prompt in task_experiments.items():
    print(f"\n【{task_name}】")
    print(f"プロンプト: {prompt}")
    print("-" * 60)
    
    result, history = generate_llada(
        model, prompt,
        steps=18, gen_length=72, block_length=36,
        temperature=0.1, cfg_scale=0.0, remasking='low_confidence'
    )
    
    task_results[task_name] = (result, history)
    
    # 即座に結果表示
    avg_conf = np.mean([h['avg_confidence'] for h in history if h['avg_confidence'] > 0])
    print(f"\n結果: {result}")
    print(f"平均信頼度: {avg_conf:.3f}")
    print("=" * 70)

In [ ]:
# タスク別性能分析とグラフ化
print("=== タスク別性能分析 ===")

# 統計データの収集
task_stats = {}
for task_name, (result, history) in task_results.items():
    avg_conf = np.mean([h['avg_confidence'] for h in history if h['avg_confidence'] > 0])
    conf_std = np.std([h['avg_confidence'] for h in history if h['avg_confidence'] > 0])
    total_steps = len(history)
    final_length = len(result.split())
    
    task_stats[task_name] = {
        'avg_confidence': avg_conf,
        'conf_std': conf_std,
        'total_steps': total_steps,
        'final_length': final_length,
        'result': result
    }

# 統計表の表示
print("\nタスク別統計:")
print("-" * 80)
print(f"{'タスク':<12} {'語数':<6} {'ステップ':<8} {'平均信頼度':<12} {'信頼度SD':<10}")
print("-" * 80)

for task_name, stats in task_stats.items():
    print(f"{task_name:<12} {stats['final_length']:<6} {stats['total_steps']:<8} "
          f"{stats['avg_confidence']:<12.3f} {stats['conf_std']:<10.3f}")

# グラフ化
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# 平均信頼度の比較
task_names = list(task_stats.keys())
confidences = [task_stats[name]['avg_confidence'] for name in task_names]
conf_stds = [task_stats[name]['conf_std'] for name in task_names]

bars1 = ax1.bar(range(len(task_names)), confidences, 
                yerr=conf_stds, capsize=5, 
                color=['#2E8B57', '#8FBC8F', '#CD853F', '#DEB887', '#708090'])
ax1.set_xlabel('タスク種別')
ax1.set_ylabel('平均信頼度')
ax1.set_title('タスク別平均信頼度')
ax1.set_xticks(range(len(task_names)))
ax1.set_xticklabels(task_names, rotation=45, ha='right')
ax1.grid(True, alpha=0.3)

# 最終語数の比較
lengths = [task_stats[name]['final_length'] for name in task_names]
bars2 = ax2.bar(range(len(task_names)), lengths,
                color=['#2E8B57', '#8FBC8F', '#CD853F', '#DEB887', '#708090'])
ax2.set_xlabel('タスク種別')
ax2.set_ylabel('生成語数')
ax2.set_title('タスク別生成語数')
ax2.set_xticks(range(len(task_names)))
ax2.set_xticklabels(task_names, rotation=45, ha='right')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n" + "=" * 80)

In [ ]:
# 各タスクの詳細な生成プロセス分析
print("=== 各タスクの生成プロセス詳細分析 ===")

for task_name, (result, history) in task_results.items():
    print(f"\n【{task_name}】の生成プロセス分析")
    print(f"最終結果: {result}")
    print("\n生成プロセス（最後の6ステップ）:")
    
    visualize_generation_with_highlights(
        history[-6:], 
        f"{task_name}の詳細プロセス",
        show_details=True
    )
    
    print("\n" + "-" * 70)

## 6. まとめと考察

In [ ]:
# 全体的な分析とまとめ
print("=== 日本語プロンプト実験の総合分析 ===")

# 1. 言語別比較の要約
print("\n1. 日本語 vs 英語の比較結果:")
print("-" * 40)

jp_confidences = []
en_confidences = []
jp_lengths = []
en_lengths = []

for (jp_result, jp_hist), (en_result, en_hist) in zip(japanese_results, english_results):
    jp_conf = np.mean([h['avg_confidence'] for h in jp_hist if h['avg_confidence'] > 0])
    en_conf = np.mean([h['avg_confidence'] for h in en_hist if h['avg_confidence'] > 0])
    
    jp_confidences.append(jp_conf)
    en_confidences.append(en_conf)
    jp_lengths.append(len(jp_result.split()))
    en_lengths.append(len(en_result.split()))

print(f"平均信頼度 - 日本語: {np.mean(jp_confidences):.3f}, 英語: {np.mean(en_confidences):.3f}")
print(f"平均語数   - 日本語: {np.mean(jp_lengths):.1f}語, 英語: {np.mean(en_lengths):.1f}語")

# 2. 日本語タスク別分析
print("\n2. 日本語タスク別性能ランキング:")
print("-" * 40)

# 信頼度でソート
sorted_tasks = sorted(task_stats.items(), key=lambda x: x[1]['avg_confidence'], reverse=True)

print("信頼度順ランキング:")
for i, (task_name, stats) in enumerate(sorted_tasks, 1):
    print(f"  {i}. {task_name}: {stats['avg_confidence']:.3f}")

# 3. 特徴的な観察結果
print("\n3. 観察された特徴:")
print("-" * 40)

highest_conf_task = sorted_tasks[0][0]
lowest_conf_task = sorted_tasks[-1][0]

print(f"• 最も安定した生成: {highest_conf_task}（信頼度: {sorted_tasks[0][1]['avg_confidence']:.3f}）")
print(f"• 最も挑戦的なタスク: {lowest_conf_task}（信頼度: {sorted_tasks[-1][1]['avg_confidence']:.3f}）")

# 4. 言語特性の分析
print("\n4. 日本語特有の生成特性:")
print("-" * 40)

if np.mean(jp_confidences) > np.mean(en_confidences):
    print("• 日本語での生成の方が高い信頼度を示している")
else:
    print("• 英語での生成の方が高い信頼度を示している")

if np.mean(jp_lengths) > np.mean(en_lengths):
    print("• 日本語の方がより詳細な応答を生成する傾向")
else:
    print("• 英語の方がより詳細な応答を生成する傾向")

print("\n5. 推奨される使用場面:")
print("-" * 40)
print(f"• 高精度が必要: {highest_conf_task}などの構造化タスク")
print(f"• 創造性が必要: {lowest_conf_task}などの表現的タスク")
print("• 実用的応用: 料理レシピや実用情報などの具体的指示")

print("\n" + "=" * 80)
print("実験完了: 日本語プロンプトでのLLaDA性能分析")
print("=" * 80)

## 実験結果サマリー

このノートブックでは、LLaDAの日本語処理能力を多角的に分析しました：

### 主要な発見

1. **言語間比較**: 日本語と英語での生成品質と特性の違い
2. **タスク別性能**: 異なる種類の日本語タスクでの相対的な性能
3. **複雑表現**: 敬語、方言、韻律など日本語特有の表現への対応
4. **信頼度パターン**: タスクタイプによる信頼度の変動

### 活用提案

- **構造化タスク**: 数学、料理レシピなど明確な手順があるタスク
- **文化的コンテンツ**: 日本特有の文化や慣習に関する説明
- **実用的情報**: 交通案内や実用的な指示

### 今後の研究方向

- より長い日本語テキストでの性能評価
- 専門分野（法律、医療など）での日本語処理
- 多様な日本語方言での比較実験